In this notebook we will try to explore the `PySINDy` package architecture, in order to easily implement our bayesian version of this algorithm with a sparsity inducing prior.

We will implement a "custom" `optimizer` module, that will implement a Maximum A Posteriori (_MAP_) algorithm over the distribution derived from data and a (possibly sparsity-inducing) prior distribution.
The main difference is that this object will retain information over the whole probability distribution for the coefficients $\boldsymbol{\xi}$ and not just the best estimates, so that we can evaluate uncertainties over such parameters.

The goal of this algorithm is to compute

$$
P(\boldsymbol{\xi}|\boldsymbol{\dot{u}},\boldsymbol{\Theta}) = P(\boldsymbol{\dot{u}}|\boldsymbol{\xi},\boldsymbol{\Theta})P(\boldsymbol{\xi})
$$

Under the assumption that the likelihood of observing $\boldsymbol{\dot{u}}$ given a certain coefficients vector $\boldsymbol{\xi}$ is a gaussian with mean given by the linear relation:

$$
P(\boldsymbol{\dot{u}}|\boldsymbol{\xi},\boldsymbol{\Theta}) \sim \mathcal{N}(\boldsymbol{\Theta}^T\boldsymbol{\xi},\sigma^2)
$$


And $P(\boldsymbol{\xi})$ will be a sparsity inducing prior, so that the original goal of finding the smallest amount of explanatory terms possible is somewhat obtained.

### Spike and Slab regression

The spike and slab regression is considered to be the golden standard of sparsity inducing priors. The main idea behind it is to build prior that's a "mixture" of some smooth other "slab" prior (such as some ridge regression gaussian pit) and a delta function (the spike), centered at some point $v$; a latent random variable $Z \sim Ber(\theta_i)$ decides which prior to use:

$$
P(\xi_i | z_i) = 0 \sim \delta(\xi_i-v) \\
P(\xi_i | z_i) = 1 \sim P_{slab}(\xi_i) 
$$

So that the prior is the slab function with probability $\theta_i$ or collapses at $v$ with probability $1-\theta_i$. Setting $v=0$ corresponds to the sparsity assumption. If we marginalize over $z_i$:

$$
P(\xi_i) = \sum_{z_i=0}^1 P(\xi_i|z_i)P(z_i) = \theta_i P_{slab}(\xi_i) + (1-\theta_i)\delta(\xi_i)
$$

If we denote with $\circ$ the Hadamard product (element-wise product), we can express the likelihood of the data given a certain "masking vector" $\boldsymbol{z}$ and a coefficients vector $\boldsymbol{\xi}$ as

$$
P(\boldsymbol{\dot{u}}|\boldsymbol{\xi},\boldsymbol{z},\boldsymbol{\Theta},\sigma^2) \sim \mathcal{N}(<\boldsymbol{z}\circ \boldsymbol{\xi},\Theta>,\sigma^2)
$$

And thus, if we simply denote the data as $\mathcal{D}$, we have the posterior

$$
P(\boldsymbol{z},\boldsymbol{\xi}|\mathcal{D}) = \frac{1}{P(\mathcal{D})} P(\boldsymbol{z}) P_{slab}(\boldsymbol{\xi})\prod_{\mathcal{D}}P(\boldsymbol{\dot{u}}|\boldsymbol{\xi},\boldsymbol{z},\boldsymbol{\Theta},\sigma^2)
$$

The evidence would need to be computed by integrating over all (infinitely many) possible values of $\boldsymbol{\xi}$ and the all the possible ($2^N$) combinations for $\boldsymbol{z}$; it is clear that an analytical derivation of this posterior distribution is infeasible and a sampling approach will need to be deployed.



#### The `BaseOptimizer` class

This is the wrapper class for each optimizer algorithm that the package provides; we will build a optimizer module as a subclass of this wrapper. <a href=https://pysindy.readthedocs.io/en/latest/_modules/pysindy/optimizers/base.html#BaseOptimizer>Source code</a> is available on the documentation.

#### Bayesian Regression implementation

The class will evaluate the best coefficients by performing a Gradient Descent on the posterior distribution, obtained by Bayes theorem with the previously presented likelihood and a prior of choice:

$$
\boldsymbol{\xi}_{best} = \argmin_{\boldsymbol{\xi}}  \left [ - P(\boldsymbol{\xi}|\boldsymbol{\boldsymbol{\dot{u}}},\boldsymbol{\Theta}) \right ] = \argmin_{\boldsymbol{\xi}} \left[ N \log{(\sigma\sqrt{2\pi})}  + \frac{\sum_i^N (\boldsymbol{\dot{u}} - \boldsymbol{\Theta}^T\boldsymbol{\xi} )^2}{2\sigma^2} - \log{P(\boldsymbol{\xi})} \right ]
$$

The initial guess for the coefficients will be the result of a OLS algorithm (already provided by the `BaseOptimizer` class)


In [3]:
from pysindy.optimizers import BaseOptimizer
import numpy as np
from sklearn.utils.validation import check_X_y
from sklearn.linear_model._base import _preprocess_data
from scipy.stats import multivariate_normal



def _rescale_data(X, y, sample_weight):
    """Rescale data so as to support sample_weight"""
    n_samples = X.shape[0]
    sample_weight = np.asarray(sample_weight)
    if sample_weight.ndim == 0:
        sample_weight = np.full(n_samples, sample_weight, dtype=sample_weight.dtype)
    sample_weight = np.sqrt(sample_weight)
    sw_matrix = sparse.dia_matrix((sample_weight, 0), shape=(n_samples, n_samples))
    X = safe_sparse_dot(sw_matrix, X)
    y = safe_sparse_dot(sw_matrix, y)
    return X, y


class SpikeSlabRegression (BaseOptimizer):
    """
    Bayesian Regression with a Spike and Slab type prior Optimizer.
    
    Computes the most likely combination of the coefficient vector
    and the masking vector using Bayesian inference to compute the 
    posterior distribution.

    Parameters
    ----------
    fit_intercept : boolean, optional (default False)
        Whether to calculate the intercept for this model. If set to false, no
        intercept will be used in calculations.
    
    return_uncertainty : boolean, optional (default False)
        ???? COSA CALCOLIAMO ???

    normalize_columns : boolean, optional (default False)
        Normalize the columns of x (the SINDy library terms) before regression
        by dividing by the L2-norm. Note that the 'normalize' option in sklearn
        is deprecated in sklearn versions >= 1.0 and will be removed.

    copy_X : boolean, optional (default True)
        If True, X will be copied; else, it may be overwritten.

    theta_prior : array, shape (n_features), optional (default 0.5 for each feature)
        Prior belief about the probability of a coefficient being zero.
        (Prior Bernoulli probabilities for the vector z)

    initial_guess_z : ....???
        HERE OR IN FIT METHOD?

    alpha : float, optional (defualt None)
        L2 penalization coefficient. This determines the strength of the prior on the coefficients.
        If None, no prior will be applied (uniform prior)
    
    ----------------------------------------???? GRADIENT DESCENT NO

    lr : float, optional (default 0.1)
        Learning rate for the gradient descent. 

    tol : float, optional (default 1e-5)
        Tolerance used for determining convergence of the optimization
        algorithm.

    max_iter : int, optional (default 100)
        Maximum iterations of the optimization algorithm.
    
    verbose : boolean, optional (default False)
        enables verbose
    
    sigma : float, optional (default 1) ----> IDEA: DEFAULT TO RESIDUAL SUM OF OLS?
        initial guess for the standard eviation of the gaussian distribution for the target value.

    Attributes
    ----------

    coef_ : array, shape (n_features,) or (n_targets,n_features)
        Coefficients vector.

    
    

    Methods
    ----------

    sample(N) :
        
    """

    def __init__(
        self, 
        max_iter=20, 
        normalize_columns=False, 
        fit_intercept=False, 
        initial_guess=None, 
        copy_X=True,
        prior='spikeslab',
        lr=0.1,
        tol=1e-5,
        max_iter=100,
        initial_guess_z=None,
        theta_prior=None,
        alpha=None,
        verbose=False
        ):

        # super() calls a temporary version of the parent class
        # this way we pass the init parameters to the class itself via inheritance
        # without having to rewrite everything
        super().__init__(max_iter, normalize_columns, fit_intercept, initial_guess, copy_X)

        self.alpha=alpha
        if initial_guess_z is None:
            self.initial_guess_z = np.ones_like(initial_guess) # initialize initial guess as all being unmasked
        if theta_prior is None:
            self.theta_prior = np.ones_like(initial_guess)/2 # 0.5 probability for each
        self.prior=prior
        self.lr=lr
        self.tol=tol
        if self.max_iter <= 0:
            raise ValueError("Max iteration must be > 0")
        self.verbose=verbose

    # WE WILL OVERRIDE THE FIT METHOD FROM BaseEstimator SINCE WE WANT DIFFERENT .ind_ attributes

    def fit(self, x_, y, sample_weight=None, **reduce_kws):
        """
        Fit the model.

        Parameters
        ----------
        x_ : array-like, shape (n_samples, n_features)
            Training data

        y : array-like, shape (n_samples,) or (n_samples, n_targets)
            Target values

        sample_weight : float or numpy array of shape (n_samples,), optional
            Individual weights for each sample

        reduce_kws : dict
            Optional keyword arguments to pass to the _reduce method
            (implemented by subclasses)

        Returns
        -------
        self : returns an instance of self
        """

        # ----------- rescaling part
        x_, y = check_X_y(x_, y, accept_sparse=[], y_numeric=True, multi_output=True)

        x, y, X_offset, y_offset, X_scale = _preprocess_data(
            x_,
            y,
            fit_intercept=self.fit_intercept,
            copy=self.copy_X,
            sample_weight=sample_weight,
        )

        if sample_weight is not None:
            x, y = _rescale_data(x, y, sample_weight)

        self.iters = 0


        # ------------ preparing dimensions, if there is only one target (only one time derivative) then we set it (-1,1) shape
        if y.ndim == 1:
            y = y.reshape(-1, 1)
        coef_shape = (y.shape[1], x.shape[1])
        self.ind_ = np.ones(coef_shape, dtype=bool)

        # ----------- normalization
        self.Theta_ = x # saving original theta
        x_normed = np.copy(x)
        if self.normalize_columns:
            reg = 1 / np.linalg.norm(x, 2, axis=0)
            x_normed = x * reg


        # ----------- initial guess via ols
        if self.initial_guess is None:
            self.coef_ = np.linalg.lstsq(x_normed, y, rcond=None)[0].T
        else:
            if not self.initial_guess.shape == coef_shape:
                raise ValueError(
                    "initial_guess shape is incompatible with training data. "
                    f"Expected: {coef_shape}. Received: {self.initial_guess.shape}."
                )
            self.coef_ = self.initial_guess

        self.history_ = [self.coef_]

        # WHERE THE MAGIC HAPPENS

        self._reduce(x_normed, y, **reduce_kws)
        #self.ind_ = np.abs(self.coef_) > 1e-14 # WE WILL SET THIS IN THE REDUCE METHOD, its gonna be the most probable z vector

        # Rescale coefficients to original units
        if self.normalize_columns:
            self.coef_ = np.multiply(reg, self.coef_)
            if hasattr(self, "coef_full_"):
                self.coef_full_ = np.multiply(reg, self.coef_full_)
            for i in range(np.shape(self.history_)[0]):
                self.history_[i] = np.multiply(reg, self.history_[i])

        self._set_intercept(X_offset, y_offset, X_scale)
        return self

    def _z_prior(self,z):
        """
        Probability of a certain vector z under the prior
        Given by vectorized Bernoulli
        """
        return np.prod(self.theta_prior**z * (1 - self.theta_prior)**(1-z))

    def _coefs_prior(self,coefs):
        if self.alpha is None:
            # uniform pseudo prior
            return 1
        else:
            # a gaussian with covariance as a diagonal identity matrix times 1/alpha
            return multivariate_normal.pdf(coefs,mean=None,cov=np.eye(len(coefs))/self.alpha)


        

    def _reduce(self,x,y):
        """
        Reduce method to actually perform the minimization.
        """

    def _posterior_unnormalized(self,x,y,coefs,z,sigma):

        likelihood = multivariate_normal.pdf()

        self._z_prior(z)*self._coef_prior(coefs)*

    
